# Учебный проект 16_Классификация комментариев магазина "Викишоп"

## Содержание

* [Описание проекта](#Описание)
* [Импорт библиотек Python](#Импорт)
* [Загрузка данных](#Загрузка)
* [Предобработка данных](#Предобработка)
* [Исследовательский анализ данных](#Исследование)
* [Вывод](#Вывод)

## Описание проекта <a class = 'anchor' id = 'Описание'></a>

На исследовании находятся данные с `информацией о комментариях на странице магазина`, которые были предоставлены руководством компании "Викишоп".

---

`Задача`

Разработать модель машинного обучения для **классификации комментариев на позитивные и негативные** с целью дальнейшей модерации токсичных комментариев. Необходимо, чтобы результат модели на метрике F1 был **не меньше 0.75**.

---

`Путь решения`

1. Собрать исторические данные о комментариях на сайте магазина;
2. Провести предобработку значений в наборе данных;
3. Построить модель BERT для анализа семантики текста;
4. Оценить качество модели BERT на метрике F1;
5. Сформировать вывод о подготовленных решениях.

---

`Располагаемые данные`

**комментарии на странице магазина - toxic_comments**
* text - содержание комментария пользователя;
* toxic - таргет: является ли комментарий токсичным.

## Импорт библиотек Python <a class = 'anchor' id = 'Импорторт'></a>

Данный блок характеризуется следующими последовательными действиями:
1. Импорт библиотек Python:
    * для манипулирования данными;
    * для визуализации данных;
    * для исследования временных рядов;
    * для решения задач машинного обучения:
        * модель линейной регрессии;
        * метрики оценки эффективности моделей;
        * механизмы отбора данных данных.
2. Инициализация переменных-констант для последующего использования на этапе построения моделей МО;
3. Формирование вывода по итогам данного этапа.

In [1]:
!pip install nltk --upgrade


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached regex-2025.11.3-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Using cached regex-2025.11.3-cp312-cp312-win_amd64.whl (277 kB)


In [4]:
# импорт библиотек python

# для манипулирования данными
import pandas as pd
import numpy as np

# для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

# для анализа и обработки текста
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# инструменты машинного обучения

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

# метрики оценки эффективности моделей
from sklearn.metrics import f1_score

# вспомогательные механизмы для задач машинного обучения
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, TimeSeriesSplit
from sklearn.utils import shuffle

# игнорирование возможных предупреждений
import warnings
warnings.filterwarnings("ignore")

# 
import os

In [3]:
# инициализация констант для дальнейшего использования в проекте
# инициализация переменной RANDOM_STATE для фиксирования случайности
# инициализация переменной TEST_SIZE для фиксирования размера тестовой выборки при разбиении наборов данных
RANDOM_STATE = 42
TEST_SIZE = 0.25

**Вывод**

1. Импортированы библиотеки Python:
    * для манипулирования данными:
        * pandas;
        * numpy.
    * для визуализации данных:
        * matplotlib.pyplot;
        * seaborn.
    * для решения задач машинного обучения:
        * модели машинного обучения:
            * LogisticRegression - модель логистической регрессии;
            * SVC - модель опорных векторов;
            * DecisionTreeClassifier - модель дерева решений;
            * RandomForestClassifier - модель случайного леса;
            * CatBoostClassifier - модель CatBoost;
        * метрики оценки эффективности моделей классификации:
            * f1_score - взвешенная метрика F1
        * механизмы анализа и обработки текста:
            * CountVectorizer - инструмент для векторизации текста на основе подсчета слов;
            * TfidfVectorizer - инструмент для векторизации текста на основе TF-IDF;
            * re - модуль для работы с регулярными выражениями;
            * nltk - библиотека для обработки естественного языка;
            * stopwords - список стоп-слов.
        * вспомогательные механизмы для задач машинного обучения:
            * train_test_split - механизм разделения данных;
            * GridSearchCV - механизм поиска гиперпараметров с перебором по "сетке";
            * cross_val_score - механизм кросс-валидации;
            * shuffle - механизм перемешивания данных.
    * для отключения предупреждений.
2. Инициализированы переменные:
    * **TEST_SIZE** для фиксирования размера тестовой выборки при разбиении наборов данных;
    * **RANDOM_STATE** для фиксирования случайности.

## Загрузка данных <a class = 'anchor' id = 'Загрузка'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Загрузка данных в рабочую среду Jupyter Notebook - инициализация переменной **comments**;
2. Вывод на экран параметров датасетов:
    * вывод общей структуры набора данных - демонстрация первых 5 строк;
    * общей информации о наборе данных.
3. Формирование вывода по итогам данного этапа.

In [5]:
# загрузка данных в рабочую среду

try:
    dataset_path = os.path.join('..', 'Datasets', 'toxic_comments.csv')
    comments = pd.read_csv(dataset_path)
except:
    taxi = pd.read_csv('/datasets/toxic_comments.csv')

In [6]:
# инициализация пользовательской функции для первичного изучения содержимого наборов данных
def first_meeting (df : pd.DataFrame, df_name : str) -> None:
    print(f'Структура набора данных {df_name}')
    display(df.head())
    print('Общая информация о наборе')
    print(df.info())
    print()

In [7]:
# вывод на экран параметров датасета 'taxi'
first_meeting(comments, 'toxic_comments')

Структура набора данных toxic_comments


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Общая информация о наборе
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None



In [8]:
# исключение столбца 'Unnamed: 0' из набора данных
comments = comments.drop('Unnamed: 0', axis=1)

**Вывод**

1. Данные загружены в рабочую среду Jupyter Notebook. Инициализирована переменная **comments**;
2. Выведена на экран общая информация о наборе данных:
    * В наборе данных **отсутствуют пустые значения**;
    * В наборе данных **159_292 строк**;
    * В наборе данных **2 столбца**:
        * text - содержание комментария пользователя;
        * toxic - таргет: является ли комментарий токсичным.
3. Столбец 'Unnamed: 0' удален из набора данных по причине повторения индексации записей.